## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from mlflow.tracking import MlflowClient


In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1751509027640, experiment_id='1', last_update_time=1751509027640, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1751508685415, experiment_id='0', last_update_time=1751508685415, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
# Load data
X, y = load_iris(return_X_y=True)

In [16]:
mlflow.set_experiment("my-experiment-1")
with mlflow.start_run():

    params = {"C": 0.5, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/03 02:38:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/07/03 02:38:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/2007347f044644549bb153ad52990839/artifacts'
🏃 View run salty-mole-896 at: http://127.0.0.1:5000/#/experiments/1/runs/2007347f044644549bb153ad52990839
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [17]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1751509027640, experiment_id='1', last_update_time=1751509027640, lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [18]:
client = MlflowClient("http://127.0.0.1:5000")

In [19]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1751509057783, description='', last_updated_timestamp=1751510114944, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1751510114944, current_stage='None', description='', last_updated_timestamp=1751510114944, name='iris-classifier', run_id='4401f099540641dbae2b067c65927d75', run_link='', source='mlflow-artifacts:/1/4401f099540641dbae2b067c65927d75/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='2'>], name='iris-classifier', tags={}>]

In [32]:
runs = client.search_runs(experiment_ids='1', order_by=["metrics.accuracy desc"], max_results=3)
for run in runs:
    # Extract model class from tags
    model_class = 'N/A'
    if 'model' in list(run.data.tags.keys()):
        model_class = run.data.tags['model']
    elif 'estimator_class' in list(run.data.tags.keys()):
        model_class = run.data.tags['estimator_class']

    print(f"Run ID: {run.info.run_id},\n \
            Status: {run.info.status},\n \
            Model: {model_class}\n \
            Accuracy: {run.data.metrics['accuracy']:.4f} \
        \n")
    pass

Run ID: a36ed9505f3247d99d084b8013bc1994,
             Status: FINISHED,
             Model: N/A
             Accuracy: 0.9733         

Run ID: c1e87ac18aba4210a7f78040420085cc,
             Status: FINISHED,
             Model: N/A
             Accuracy: 0.9733         

Run ID: a2e4bc8463644160b6bfb0ede5ce3018,
             Status: FINISHED,
             Model: N/A
             Accuracy: 0.9733         



In [33]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
run_id = 'a36ed9505f3247d99d084b8013bc1994'
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2025/07/03 02:45:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 4
Created version '4' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1751510752248, current_stage='None', description='', last_updated_timestamp=1751510752248, name='iris-classifier', run_id='a36ed9505f3247d99d084b8013bc1994', run_link='', source='mlflow-artifacts:/1/a36ed9505f3247d99d084b8013bc1994/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [21]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1751509057783, description='', last_updated_timestamp=1751510301550, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1751510301550, current_stage='None', description='', last_updated_timestamp=1751510301550, name='iris-classifier', run_id='2007347f044644549bb153ad52990839', run_link='', source='mlflow-artifacts:/1/2007347f044644549bb153ad52990839/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='3'>], name='iris-classifier', tags={}>]

In [ ]:
name='iris-classifier'
alias = 'Testing'
version = '4'
client.set_registered_model_alias(name=name, alias=alias, version=version)